## Tutorial for using Prompt flow and sending traces to Arize AI

Import arize_otel and other dependencies required for setup.

In [ ]:
import os
from arize_otel import register_otel, Endpoints
from opentelemetry.sdk.environment_variables import OTEL_EXPORTER_OTLP_ENDPOINT
from promptflow.tracing._start_trace import setup_exporter_from_environ

Set up OpenTelemetry using our convenience function: register_otel with your Arize credentials.

In [ ]:
# Setup OTEL via our convenience function
register_otel(
    endpoints=Endpoints.ARIZE,
    space_key="INSERT_YOUR_ARIZE_SPACE_KEY_HERE",  # in app space settings page
    api_key="INSERT_YOUR_ARIZE_API_KEY_HERE",  # in app space settings page
    model_id="test-prompt-flow-chat",  # name this to whatever you would like
)

Then set up the OpenTelemetry endpoint to be Arize and use Prompt flow's setup_exporter_from_environ to start tracing any further flows and LLM calls.

In [ ]:
os.environ[OTEL_EXPORTER_OTLP_ENDPOINT] = Endpoints.ARIZE
setup_exporter_from_environ()

Open and read flow.py which contains ChatFlow, the flow we are using in this example.

In [ ]:
with open("flow.py") as fin:
    print(fin.read())

Create your Azure OpenAI Connection and follow the link in the comments if you need to create an account. If not, paste your credentials where specified.

In [ ]:
from promptflow.client import PFClient
from promptflow.connections import AzureOpenAIConnection

# client can help manage your runs and connections.
pf = PFClient()

# Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure OpenAI resource.
connection = AzureOpenAIConnection(
    name="open_ai_connection",
    api_key="INSERT_YOUR_API_KEY_HERE",
    api_base="INSERT_YOUR_API_BASE_HERE",
    api_type="azure",
)

conn = pf.connections.create_or_update(connection)
print("successfully created connection")

Create your config with a specified model to be used in flow calls

In [ ]:
from promptflow.core import AzureOpenAIModelConfiguration

# create the model config to be used in below flow calls
config = AzureOpenAIModelConfiguration(connection="open_ai_connection", azure_deployment="gpt-4o")

Create your Chat Flow and see the traces rendered in Arize!

In [ ]:
from flow import ChatFlow

# create a chatFlow obj with connection
chat_flow = ChatFlow(config)
# run the flow as function, which will be recorded in the trace
result = chat_flow(question="What is ChatGPT? Please explain with concise statement")
result